In [2]:
import cv2 as cv
import pandas as pd
import tensorflow as tf
import numpy as np
import os

In [3]:
haar_cascade=cv.CascadeClassifier(cv.data.haarcascades + "haarcascade_frontalface_default.xml")
emotions = ['happy','sad']
DIR = 'emotiondataset/data'


def create_train():
    features = []
    labels = []
    for person in emotions:
        path = f"{DIR}/{person}"
        label ="happy" if emotions.index(person)==0 else "sad"

        for img in os.listdir(path):
            img_path = f"{path}/{img}"
            if(img_path is None):
                continue
            image_seq=cv.imread(img_path)
            
            
            gray=cv.cvtColor(image_seq,cv.COLOR_BGR2GRAY)
            faces_rect = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
            if len(faces_rect)<=0 or faces_rect is None:
                continue
            for (x,y,w,h) in faces_rect:
                faces_roi =cv.resize(image_seq[y:y+h, x:x+w],(224,224))
                  
                features.append(faces_roi)
                labels.append(label)
            
            
            

            
    return np.array(features,dtype="float32")/255.0,np.array(labels)

features,labels=create_train()
print('Training done ------------dd---')
print(features)
print(labels)

Training done ------------dd---
[[[[0.36862746 0.21960784 0.2       ]
   [0.36862746 0.21960784 0.2       ]
   [0.36862746 0.21568628 0.19607843]
   ...
   [0.34117648 0.18039216 0.24313726]
   [0.34117648 0.18039216 0.23921569]
   [0.34117648 0.18039216 0.23921569]]

  [[0.36862746 0.21960784 0.2       ]
   [0.36862746 0.21960784 0.2       ]
   [0.36862746 0.21568628 0.19607843]
   ...
   [0.34509805 0.18039216 0.24313726]
   [0.34117648 0.18039216 0.23921569]
   [0.34117648 0.18039216 0.23921569]]

  [[0.36862746 0.21568628 0.2       ]
   [0.36862746 0.21568628 0.2       ]
   [0.36862746 0.21568628 0.19607843]
   ...
   [0.3372549  0.18039216 0.24705882]
   [0.3372549  0.18431373 0.24705882]
   [0.3372549  0.18431373 0.24705882]]

  ...

  [[0.3529412  0.2901961  0.42352942]
   [0.3529412  0.2901961  0.42352942]
   [0.3529412  0.2901961  0.42745098]
   ...
   [0.23921569 0.22352941 0.3882353 ]
   [0.22352941 0.20392157 0.37254903]
   [0.22352941 0.20392157 0.37254903]]

  [[0.3607843

In [4]:
np.unique(labels)

array(['happy', 'sad'], dtype='<U5')

In [5]:
labels=pd.get_dummies(labels)
labels

,happy,sad
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
207,False,True
208,False,True
209,False,True
210,False,True


In [6]:
data=tf.data.Dataset.from_tensor_slices((tf.constant(features),tf.constant(labels))).shuffle(buffer_size=len(features)).batch(8)
data

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.bool, name=None))>

In [17]:
def create_model():
    model=tf.keras.Sequential(
        [
            tf.keras.Input(shape=(224, 224, 3)),
            tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
        
            tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
            
            tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
        
            tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'),

            tf.keras.layers.MaxPool2D(pool_size=(2, 2)),
            tf.keras.layers.BatchNormalization(),

            tf.keras.layers.Flatten(),
            tf.keras.layers.Dropout(0.5),  # Dropout oranı artırıldı
            tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.L1L2(l1=1e-5, l2=1e-4)),
            tf.keras.layers.BatchNormalization(),
            

            tf.keras.layers.Dense(2, activation='softmax')  # Çıkış katmanı (iki sınıf için)
        ]
    )
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(),
        metrics=["accuracy"]
    )
    return model
def train_model():
    model=create_model()
    model.fit(data,epochs=15)
    return model


In [18]:
def train_model():
    model=create_model()
    model.fit(data,epochs=35)
    return model

In [19]:
model=train_model()

Epoch 1/35
27/27 [==============================] - 10s 354ms/step - loss: 1.8925 - accuracy: 0.6462
Epoch 2/35
27/27 [==============================] - 9s 348ms/step - loss: 1.4619 - accuracy: 0.7170
Epoch 3/35
27/27 [==============================] - 9s 346ms/step - loss: 1.2978 - accuracy: 0.7594
Epoch 4/35
27/27 [==============================] - 9s 348ms/step - loss: 1.1977 - accuracy: 0.8113
Epoch 5/35
22/27 [=======================>......] - ETA: 1s - loss: 1.1062 - accuracy: 0.8352

KeyboardInterrupt: 

In [16]:
def live_train():
    video = cv.VideoCapture(0)
    if not video.isOpened():
        print("Kamera açılamadı!")
        return

    while True:
        isTrue, frame = video.read()
        if not isTrue or frame is None:
            print("Kare alınamadı!")
            continue

        gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
        face_rect = haar_cascade.detectMultiScale(gray, minNeighbors=3, scaleFactor=1.1)

        for (x, y, w, h) in face_rect:
            if w > 0 and h > 0:
                faces_roi = cv.resize(frame[y:y+h, x:x+w], (224, 224))
                faces_roi = faces_roi.astype("float32") / 255.0
                pred = model.predict(np.expand_dims(faces_roi, axis=0))

                cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), thickness=2)
                cv.putText(frame, str(emotions[np.argmax(pred)]), (x, y - 10), 
                           cv.FONT_HERSHEY_COMPLEX, 1.0, (0, 255, 0), thickness=2)

        cv.imshow("Video", frame)
        if cv.waitKey(20) & 0xFF == ord("d"):
            break

    video.release()
    cv.destroyAllWindows()

live_train()

1/1 [==============================] - 0s 36ms/step
